In [1]:
import numpy as np

In [2]:
def trace_distance(prob_dist_1, prob_dist_2):
    prob_dist_1 = np.array(prob_dist_1)
    prob_dist_2 = np.array(prob_dist_2)

    if prob_dist_1.shape != prob_dist_2.shape:
        raise ValueError("The two probability distributions must have the same shape.")
    
    trace_dist = 0.5 * np.sum(np.abs(prob_dist_1 - prob_dist_2))
    
    return trace_dist

In [3]:
def fidelity(rho1, rho2):
    if isinstance(rho1, DensityMatrix):
        rho1 = rho1.data
    if isinstance(rho2, DensityMatrix):
        rho2 = rho2.data

    if rho1.shape != rho2.shape:
        raise ValueError("The two density matrices must have the same shape.")

    sqrt_rho1 = np.linalg.cholesky(rho1 + 1e-10 * np.eye(rho1.shape[0])) 
    intermediate = sqrt_rho1 @ rho2 @ sqrt_rho1
    sqrt_intermediate = np.linalg.sqrtm(intermediate)
    fidelity_value = np.real(np.trace(sqrt_intermediate)) ** 2

    return fidelity_value

In [4]:
def bures_distance(rho1, rho2):
    if isinstance(rho1, DensityMatrix):
        rho1 = rho1.data
    if isinstance(rho2, DensityMatrix):
        rho2 = rho2.data

    if rho1.shape != rho2.shape:
        raise ValueError("The two density matrices must have the same shape.")

    sqrt_rho1 = np.linalg.cholesky(rho1 + 1e-10 * np.eye(rho1.shape[0]))  # Add small noise for stability
    intermediate = sqrt_rho1 @ rho2 @ sqrt_rho1
    sqrt_intermediate = np.linalg.sqrtm(intermediate)
    fidelity_value = np.real(np.trace(sqrt_intermediate)) ** 2
    bures_dist = np.sqrt(2 * (1 - np.sqrt(fidelity_value)))

    return bures_dist

In [5]:
def quantum_relative_entropy(rho1, rho2):
    if isinstance(rho1, DensityMatrix):
        rho1 = rho1.data
    if isinstance(rho2, DensityMatrix):
        rho2 = rho2.data
        
    if rho1.shape != rho2.shape:
        raise ValueError("The two density matrices must have the same shape.")

    if not np.allclose(np.trace(rho1), 1) or not np.allclose(np.trace(rho2), 1):
        raise ValueError("Both rho1 and rho2 must have trace equal to 1.")
    if np.any(np.linalg.eigvalsh(rho1) < 0) or np.any(np.linalg.eigvalsh(rho2) < 0):
        raise ValueError("Both rho1 and rho2 must be positive semidefinite.")

    log_rho1 = np.zeros_like(rho1)
    log_rho2 = np.zeros_like(rho2)

    eigenvalues1, eigenvectors1 = np.linalg.eigh(rho1)
    eigenvalues2, eigenvectors2 = np.linalg.eigh(rho2)

    log_rho1 = eigenvectors1 @ np.diag(np.log(eigenvalues1 + 1e-10)) @ eigenvectors1.T.conj()
    log_rho2 = eigenvectors2 @ np.diag(np.log(eigenvalues2 + 1e-10)) @ eigenvectors2.T.conj()

    relative_entropy = np.real(np.trace(rho1 @ (log_rho1 - log_rho2)))

    return relative_entropy

In [6]:
def hellinger_distance(rho1, rho2):
    if isinstance(rho1, DensityMatrix):
        rho1 = rho1.data
    if isinstance(rho2, DensityMatrix):
        rho2 = rho2.data

    if rho1.shape != rho2.shape:
        raise ValueError("The two density matrices must have the same shape.")

    sqrt_rho1 = np.linalg.cholesky(rho1 + 1e-10 * np.eye(rho1.shape[0]))  # Add small noise for stability
    intermediate = sqrt_rho1 @ rho2 @ sqrt_rho1
    sqrt_intermediate = np.linalg.sqrtm(intermediate)
    trace_value = np.real(np.trace(sqrt_intermediate))
    hellinger_dist = np.sqrt(1 - trace_value)

    return hellinger_dist

In [7]:
def quantum_wasserstein_distance(rho1, rho2):

    if isinstance(rho1, DensityMatrix):
        rho1 = rho1.data
    if isinstance(rho2, DensityMatrix):
        rho2 = rho2.data

    if rho1.shape != rho2.shape:
        raise ValueError("The two density matrices must have the same shape.")

    if not np.allclose(np.trace(rho1), 1) or not np.allclose(np.trace(rho2), 1):
        raise ValueError("Both rho1 and rho2 must have trace equal to 1.")
    if np.any(np.linalg.eigvalsh(rho1) < 0) or np.any(np.linalg.eigvalsh(rho2) < 0):
        raise ValueError("Both rho1 and rho2 must be positive semidefinite.")

    sqrt_rho1 = np.linalg.cholesky(rho1 + 1e-10 * np.eye(rho1.shape[0]))  # Stabilize numerical issues
    intermediate = sqrt_rho1 @ rho2 @ sqrt_rho1
    sqrt_intermediate = np.linalg.sqrtm(intermediate)
    fidelity = np.real(np.trace(sqrt_intermediate)) ** 2

    wasserstein_distance = np.sqrt(1 - fidelity)

    return wasserstein_distance

In [8]:
def von_neumann_entropy(rho):
    eigenvalues = np.linalg.eigvalsh(rho)
    eigenvalues = np.clip(eigenvalues, 0, 1)
    entropy = -np.sum(eigenvalues * np.log(eigenvalues + 1e-10))  # Add small value for stability
    return entropy

def quantum_jensen_shannon_divergence(rho1, rho2):
    if isinstance(rho1, DensityMatrix):
        rho1 = rho1.data
    if isinstance(rho2, DensityMatrix):
        rho2 = rho2.data

    if rho1.shape != rho2.shape:
        raise ValueError("The two density matrices must have the same shape.")

    S_rho1 = von_neumann_entropy(rho1)
    S_rho2 = von_neumann_entropy(rho2)

    rho_avg = 0.5 * (rho1 + rho2)

    S_avg = von_neumann_entropy(rho_avg)

    qjsd = S_avg - 0.5 * (S_rho1 + S_rho2)

    return qjsd

In [9]:
def hilbert_schmidt_distance(rho1, rho2):
    if isinstance(rho1, DensityMatrix):
        rho1 = rho1.data
    if isinstance(rho2, DensityMatrix):
        rho2 = rho2.data

    if rho1.shape != rho2.shape:
        raise ValueError("The two density matrices must have the same shape.")

    delta_rho = rho1 - rho2

    distance = np.sqrt(np.trace(delta_rho.conj().T @ delta_rho).real)

    return distance